In [9]:
# id with 0 = depot
nodes = [
    { 'id': 0, 'latitude': 4.4184, 'longitude': 114.0932, 'demand': 0 },
    { 'id': 1, 'latitude': 4.3555, 'longitude': 113.9777, 'demand': 5 },
    { 'id': 2, 'latitude': 4.3976, 'longitude': 114.0049, 'demand': 8 },
    { 'id': 3, 'latitude': 4.3163, 'longitude': 114.0764, 'demand': 3 },
    { 'id': 4, 'latitude': 4.3184, 'longitude': 113.9932, 'demand': 6 },
    { 'id': 5, 'latitude': 4.4024, 'longitude': 113.9932, 'demand': 5 },
    { 'id': 6, 'latitude': 4.4142, 'longitude': 113.9896, 'demand': 8 },
    { 'id': 7, 'latitude': 4.4804, 'longitude': 114.0127, 'demand': 3 },
    { 'id': 8, 'latitude': 4.3818, 'longitude': 114.0734, 'demand': 6 },
    { 'id': 9, 'latitude': 4.4935, 'longitude': 114.2034, 'demand': 5 },
    { 'id': 10, 'latitude': 4.4932, 'longitude': 114.1322, 'demand': 8 },
]

vehicles = [
    { 'vehicle': 'A', 'maximum_capacity': 25, 'cost': 1.2 },
    { 'vehicle': 'B', 'maximum_capacity': 30, 'cost': 1.5 }
]

In [297]:
import math
import networkx as nx
import matplotlib.pyplot as plt

class Node:
    def __init__(self, id, latitude, longitude, demand, index):
        self.id = id
        self.latitude = latitude
        self.longitude = longitude
        self.demand = demand
        self.index = index

    def calculate_distance(self, node):
        return 100 * math.sqrt(
            math.pow(node.longitude - self.longitude, 2) +
            math.pow(node.latitude - self.latitude, 2)
        )
  
class Vehicle:
    def __init__(self, vehicle, maximum_capacity, cost):
        self.vehicle = vehicle
        self.maximum_capacity = maximum_capacity
        self.cost = cost

class Path:
    def __init__(self): 
        self.nodes = []

    def add_node(self, node):
        self.nodes.append(node)

    def calculate_total_distance(self):
        total_distance = 0
        for i in range(len(self.nodes) - 1):
            total_distance += self.nodes[i].calculate_distance(self.nodes[i+1])

        return total_distance
  
    def print_path(self):
        for index, node in enumerate(self.nodes):
            if index != len(self.nodes) - 1:
                print(node.id, end=' -> ')
            else:
                print(node.id)
                print()

    def print_output(self):
        pass

    def plot_path(self, figsize=(14, 8)):
        """
            Visualize the path
            Placement of label is not consistent 
        """
        G = nx.DiGraph()
        edges = []
        for index in range(len(self.nodes) - 1):
            edges.append(
            (
                self.nodes[index].id, 
                self.nodes[index+1].id, 
                {'weight': round(self.nodes[index].calculate_distance(self.nodes[index+1]), 4)} # weight between 2 nodes
            )
            )

        G.add_edges_from(edges)
        plt.figure(figsize=figsize)
        pos = nx.circular_layout(G)
        nx.draw(G, with_labels=True)
        edge_weight = nx.get_edge_attributes(G,'weight')
        nx.draw_networkx_edge_labels(G, pos, edge_labels = edge_weight)
        plt.show()

In [412]:
import numpy as np
import random

def calculate_customer_heuristics(
    vehicle_k, # vehicle type (object)
    vertice, # customer vertice
    partitioned_vertices, # all customers of the same type of fleet
    vehicle_pheromone,
    beta,
):
    """
        Calculate heuristics for the customer allocation's state transition rules
        Calculates ratio of capacity to fixed cost
        Caclulate shortest distance among all edges between the customers of same vehicle type
    """
    distances = []
    for i in range(len(partitioned_vertices)):
        # calculate all distance between the customer and other customers in the same vehicle type
        distances.append(vertice.calculate_distance(partitioned_vertices[i]))

    # get the minimum distance and calculate the heuristic
    minimum_distance = np.min(distances)
    heuristic = (vehicle_k.maximum_capacity / vehicle_k.cost) + (1 / minimum_distance)
    heuristic = vehicle_pheromone * (math.pow(heuristic, beta))

    return heuristic

def allocate_customer(
    vertices, # all nodes
    vehicles,
    vehicle_pheromonme,
    q0, # exploitation or exploration
):
    """
        Allocate customer to each type of vehicle
        First customer is assigned randomly
        Remaining customer is assigned based on the state transition rule
        State rule: (Pheromone x heuristics)
    """
    BETA = 1 # determine whether the cost and distance is more important than pheromone or not 

    assigned_vertices = [False] * len(vertices)
    partitioned_vertices = {}
    for index in range(len(vehicles)):
        # randomly select first customer and assign to each type of vehicle
        random_index = random.randint(1, len(vertices) - 1) # skip depot
        assigned_vertices[random_index] = True # mark it as assigned
        partitioned_vertices[index] = [vertices[0], vertices[random_index]] # add depot along with that random index


    for vertices_index in range(1, len(vertices)):
        heuristics = []

        # calculate the customer heuristics and select a node to assign
        if assigned_vertices[vertices_index]:
            # if the customer is already assigned
            heuristics.append(0) # make the model not selecting assigned node

        else:
            # if the customer is not assigned to any vehicle yet
            for vehicle_index in range(len(vehicles)):
                # calculate heuristics for each vehicle for customer i

                heuristics.append(
                    calculate_customer_heuristics(
                        vehicles[vehicle_index],
                        vertices[vertices_index],
                        partitioned_vertices[vehicle_index],
                        vehicle_pheromonme[vertices_index][vehicle_index],
                        BETA
                    )
                )

            # assign the customer based on which heuristic is higher
            q = np.random.uniform(0, 1)
            if q <= q0:
                # select based on best heuristics (exploitation)
                vehicle_assigned = np.argmax(heuristics)

            else:
                # select based on probability distribution (exploration)
                np_heuristics = np.array(heuristics)
                probs = np_heuristics / np_heuristics.sum()
                vehicle_assigned = np.random.choice(np.arange(np_heuristics.size), p=probs)

            partitioned_vertices[vehicle_assigned].append(vertices[vertices_index])
            assigned_vertices[vertices_index] = True

    return partitioned_vertices
    

def update_global_pheromone_cust(
    vehicle_pheromone,
    global_best_distance,
    local_best_distance,
    local_worst_distance,
    evaporation_coefficient,
):
    """
        Update pheromone based on the best solution so far and best solution in this iteration
    """
    delta = (local_worst_distance - global_best_distance) + (local_worst_distance - local_best_distance)
    vehicle_pheromone = ((1 - evaporation_coefficient) * vehicle_pheromone) + (evaporation_coefficient * delta)
    

def update_local_pheromone_cust(
    vehicle_pheromone,
    evaporation_coefficient,
    initial_values
):
    """
        Update local pheromone matrix for customer allocation stage
    """
    vehicle_pheromone = (vehicle_pheromone * (1 - evaporation_coefficient)) + (evaporation_coefficient * initial_values)

    return vehicle_pheromone 

def calculate_route_heuristics(
    pheromone, # pheromone level for that route
    depot_node,
    source_node,
    dst_node,
    beta, # coefficient for n
):
    """
        Calculate heuristics for route construction
    """
    G = 2
    F = 2

    intermediate_result = source_node.calculate_distance(depot_node) + depot_node.calculate_distance(dst_node)
    n = intermediate_result - (G * source_node.calculate_distance(dst_node)) + (F * abs(intermediate_result))

    return pheromone * (math.pow(n, beta))


def decide_next_node(
    vertices, # all vertices available for the type of fleet (including depot)
    pheromone_matrix,
    visited_nodes,
    current_node,
    q0,
):
    """
        Decide next node based on the probability calculated 
    """
    BETA = 1

    heuristic_nodes = []

    for next_node in range(len(vertices)):
        if next_node == 0 or visited_nodes[next_node]:
            # if either visited nodes 
            heuristic_nodes.append(0.0) # make the model not selecting visited node

        else:
            route_heuristic = calculate_route_heuristics(
                pheromone_matrix[vertices[current_node].index][vertices[next_node].index], # pheromone for that edge
                vertices[0], # depot
                vertices[current_node], # node i
                vertices[next_node], # node j
                BETA
            )

            heuristic_nodes.append(route_heuristic)

    print("heuristic nodes", heuristic_nodes)
    q = np.random.uniform(0, 1)
    if q <= q0:
        # exploitation
        next_node = np.argmax(heuristic_nodes)

    else:
        # exploration
        np_heuristics = np.array(heuristic_nodes)
        probs = np_heuristics / np_heuristics.sum()
        print("np heuristics", np_heuristics)
        next_node = np.random.choice(np.arange(np_heuristics.size), p=probs)
        

    return next_node

def update_global_pheromone_route(
    pheromone, # existing pheromone
    global_best_distance, 
    local_best_distance,
    third_distance,
    pheromone_coefficient 
):
    """
        Update global pheromone matrix for route construction
    """
    delta_pheromone = ((third_distance - global_best_distance) + (third_distance - local_best_distance)) / third_distance
    new_pheromone = ((1 - pheromone_coefficient) * (pheromone)) + (pheromone_coefficient * delta_pheromone)
    
    return new_pheromone

def update_local_pheromone_route(
    pheromone,
    evaporation_coefficient, # user-defined parameter
    initial_values # user-defined parameter
):
    """
        Update local pheromone for route construction (evaporation)
    """
    pheromone_matrix = pheromone + (evaporation_coefficient * initial_values)

    return pheromone_matrix

def main(
    vertices,
    vehicles,
    n_iters,
    n_ants,
    pheromone_coefficient, # between 0 and 1
    evaporation_coefficient, # between 0 and 1
    initial_pheromone_value,
    q0,
):
    vehicle_pheromone = np.full((len(vertices), len(vehicles)), initial_pheromone_value)
    route_pheromone = np.full((len(vertices), len(vertices)), initial_pheromone_value)
    global_best_path = None

    for _ in range(n_iters):
        local_best_path = None 
        for _ in range(n_ants):
            
            # construct hvrp solution (customer allocation + route construction)
            assigned_customers = allocate_customer(
                vertices,
                vehicles,
                vehicle_pheromone,
                q0
            )

            for index in range(len(vehicles)):
                nodes_for_vehicle_k = assigned_customers[index]

                # route construction
                visited_nodes = [False] * len(nodes_for_vehicle_k)
                visited_nodes[0] = True
                current_node = 0
                current_capacity = 0

                # start from depot
                route = Path()
                route.add_node(nodes_for_vehicle_k[0])
                
                while not all(visited_nodes):
                    next_node = decide_next_node(
                        nodes_for_vehicle_k,
                        route_pheromone,
                        visited_nodes,
                        current_node,
                        q0
                    )

                    if current_capacity + assigned_customers[index][next_node].demand > vehicles[index].maximum_capacity:
                        # if the demand is more than the capacity
                        next_node = 0 # back to depot
                        current_capacity = 0 # free all load 

                    else:
                        # mark the node as visited and add to the path
                        visited_nodes[next_node] = True
                        current_capacity += nodes_for_vehicle_k[next_node].demand
                    
                    route.add_node(nodes_for_vehicle_k[next_node])
                    current_node = next_node

                current_node = 0
                route.add_node(nodes_for_vehicle_k[current_node])
                route.print_path()


            # update local pheromone update based on the solution constructed

            # record local best solution

        # insertion move and swap move

        # compare the local best solution with the global best solution, then update global best solution

        # update global pheromone

    # return global best solution
    


In [311]:
vertices_objs = [Node(**node, index=index) for index, node in enumerate(nodes)]
vehicles_objs = [Vehicle(**vehicle) for vehicle in vehicles]
vehicle_pheromone = np.ones((len(vertices_objs), len(vehicles_objs)))

test = allocate_customer(vertices_objs, vehicles_objs, vehicle_pheromone, 0.5)
print([node.id for node in test[0]])
print([node.id for node in test[1]])

[0, 6, 1, 2, 3, 4, 5, 7, 8, 10]
[0, 9]


In [392]:
route_pheromone = np.ones((len(vertices_objs), len(vertices_objs)))
decide_next_node(vertices_objs, route_pheromone, [False] * len(vertices_objs), 0, 0.5)

heuristic nodes [0.0, 13.151676699189105, 9.071675699669925, 10.347294332336185, 14.142135623730587, 10.12719112093717, 10.368510018319906, 10.160831658875244, 4.16124981225542, 13.335685209242653, 8.43566239248589]
np heuristics [ 0.         13.1516767   9.0716757  10.34729433 14.14213562 10.12719112
 10.36851002 10.16083166  4.16124981 13.33568521  8.43566239]


2

In [413]:
vertices_objs = [Node(**node, index=index) for index, node in enumerate(nodes)]
vehicles_objs = [Vehicle(**vehicle) for vehicle in vehicles]

main(
    vertices_objs,
    vehicles_objs,
    1,
    1,
    0.2,
    0.2,
    1.0,
    0.5
)

heuristic nodes [0.0, 13.335685209242653, 9.071675699669925, 14.142135623730587, 10.12719112093717, 10.368510018319906, 10.160831658875244, 4.16124981225542, 8.43566239248589]
3
heuristic nodes [0.0, 27.718181269632293, 53.62952590151699, 0.0, 56.00798023400317, 54.358413514947, 40.275023745452756, 34.46354289644084, 23.06750549821357]
4
heuristic nodes [0.0, 24.57018292348034, 55.06733179281743, 0.0, 0.0, 59.0197161356443, 44.78395639952862, 26.304646484884834, 22.482753819155597]
np heuristics [ 0.         24.57018292 55.06733179  0.          0.         59.01971614
 44.7839564  26.30464648 22.48275382]
5
heuristic nodes [0.0, 25.506037907175156, 53.805471329756024, 0.0, 0.0, 0.0, 47.5651152051621, 25.620194929621206, 23.80837025691722]
np heuristics [ 0.         25.50603791 53.80547133  0.          0.          0.
 47.56511521 25.62019493 23.80837026]
2
heuristic nodes [0.0, 13.335685209242653, 9.071675699669925, 0.0, 0.0, 0.0, 10.160831658875244, 4.16124981225542, 8.43566239248589]
n